# Hackaton 05.09.2022 - 26.09.2022
## Motorica Gesture Classification Task

![hand](hand-145_1-2.jpeg)

### Задача классификации жестов на основе данных, полученных от оптомиографических датчиков
### Ментор — Андрей Рысистов


## Description

Добрый день, участниками и участницам!

Вас приветствует компания Моторика. Мы изготавливаем и помогаем получить бионические протезы людям с врожденными особенностями и ампутациями на уровне кисти и предплечья.

Совместно с SkillFactory мы проводим данный хакатон.

Для улучшения качества работы наших протезов мы решаем задачу “распознавания жестов”.
В данном хакатоне мы предлагаем решить задачу классификации с известными заранее классами:

* сгиб/разгиб большого пальца;
* сгиб/разгиб указательного пальца;
* сгиб/разгиб среднего пальца;
* сгиб/разгиб безымянного пальца;
* сгиб/разгиб мизинца;
* сгиб/разгиб жеста "grab" (все пальцы);
* сгиб/разгиб жеста "ok" (большой и указательный палец);
* сгиб/разгиб жеста "пистолет" (средний, безымянный и мизинец);
* Задача: построение классификационной модели, которая по показаниям от датчиков будет определять жест, выполненный пользователем протеза во время записи данных, что отражено в значении целевой переменной.

## Evaluation

Метрика оценки качества решения
В качестве метрики оценки качества решения выбран Mean F-score.

Целевые значения будут использованы для валидации решений участников на лидерборде. 50% целевых значений будут использоваться на открытом лидерборде, 50% на закрытом (участники не видят его до конца соревнования) во избежания переобучения “под лидерборд”. Победитель определяется по результатам на закрытом лидерборде.

Формат решения
Формат решения можно посмотреть в файле sample_submission.csv.

In [26]:
# Import all nessesary libraries
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [6]:
# Load the data
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')

In [7]:
X_train.shape

(165, 40, 60)

In [12]:
Y_train = pd.read_csv('y_train.csv')
Y_train.head()

,Id,Class
0,0,5
1,1,8
2,2,11
3,3,9
4,4,14


Transfrom the 3D numpy array to the list of lists of pandas dataframes

Class 0                 Class 1                 Class 15

[[DF1, DF2, ... DFn], [DF1, DF2, ... DFm], .....[DF1, DF2, ... DFk]]

In [36]:
n_samples = X_train.shape[0]
n_classes = 16
data = list()
for i in range(0, n_classes):
    data.append(list())


for i in range(0, n_samples):
    #r_scaler = preprocessing.RobustScaler()
    idx = Y_train['Class'].iloc[i]
    #sample = r_scaler.fit_transform(X_train[i, :, :].transpose())
    sample = X_train[i, :, :].transpose()
    data[idx].append(pd.DataFrame(sample))
    
display(data[3][0].head(3))
display(data[15][0].head(3))

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,870.0,8.0,724.0,24.0,21.0,796.0,88.0,6.0,824.0,6.0,...,2308.0,76.0,17.0,1198.0,137.0,25.0,1671.0,24.0,872.0,24.0
1,870.0,5.0,724.0,19.0,14.0,797.0,88.0,3.0,824.0,7.0,...,2310.0,72.0,14.0,1200.0,137.0,24.0,1670.0,25.0,874.0,22.0
2,870.0,11.0,729.0,24.0,18.0,797.0,91.0,5.0,824.0,7.0,...,2313.0,69.0,16.0,1201.0,135.0,25.0,1668.0,28.0,870.0,21.0


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,832.0,13.0,1146.0,17.0,24.0,1529.0,41.0,7.0,833.0,14.0,...,1045.0,78.0,13.0,1119.0,113.0,23.0,1747.0,24.0,402.0,23.0
1,841.0,12.0,1150.0,18.0,26.0,1542.0,42.0,8.0,844.0,9.0,...,1046.0,72.0,14.0,1116.0,115.0,21.0,1748.0,26.0,402.0,22.0
2,850.0,13.0,1156.0,21.0,27.0,1562.0,40.0,2.0,851.0,9.0,...,1045.0,34.0,16.0,1117.0,101.0,23.0,1745.0,32.0,408.0,23.0


In [40]:
gestures = list()

for sample_class in data:
    gesture = 0
    n_samples = 0
    for sample in sample_class:
        gesture += sample
        n_samples += 1
    gestures.append(gesture/n_samples)

In [42]:
print(gestures[0])

             0          1            2          3          4            5   \
0   1400.090909  10.545455  1800.181818  39.545455  30.000000  2130.636364   
1   1402.181818  10.727273  1800.272727  40.636364  29.818182  2130.272727   
2   1404.272727  13.454545  1798.909091  38.818182  29.727273  2129.636364   
3   1407.000000  11.363636  1795.363636  41.454545  32.818182  2128.636364   
4   1408.818182  11.272727  1791.090909  38.000000  30.454545  2127.727273   
5   1408.909091   7.181818  1789.636364  37.000000  31.545455  2125.909091   
6   1409.363636  10.000000  1786.818182  38.363636  29.909091  2125.181818   
7   1411.090909   7.090909  1784.727273  37.545455  30.090909  2125.636364   
8   1414.181818   9.818182  1783.727273  38.181818  33.000000  2126.363636   
9   1415.363636   8.909091  1783.000000  38.818182  31.090909  2128.000000   
10  1418.181818   9.818182  1783.000000  37.363636  31.181818  2129.545455   
11  1420.636364  10.454545  1782.454545  39.727273  31.545455  2

In [ ]:
#Создаем объект класса случайный лес
rf_clf_2d = ensemble.RandomForestClassifier(
    n_estimators=500, #число деревьев
    criterion='entropy', #критерий эффективности
    max_depth=3, #максимальная глубина дерева
    max_features='sqrt', #число признаков из метода случайных подространств
    random_state=42 #генератор случайных чисел
)
#Обучаем модель 
rf_clf_2d.fit(X, y)

#Делаем предсказание класса
y_pred = rf_clf_2d.predict(X)
#Выводим отчет о метриках
print(metrics.classification_report(y, y_pred))